In [24]:
import pandas as pd
genuine_accounts_tweets = pd.read_csv('cresci-2017.csv/datasets_full.csv/genuine_accounts.csv/tweets.csv', low_memory=False)
traditional_spambots_1_tweets = pd.read_csv('cresci-2017.csv/datasets_full.csv/traditional_spambots_1.csv/tweets.csv', low_memory=False)
fake_followers_tweets=pd.read_csv('cresci-2017.csv/datasets_full.csv/fake_followers.csv/tweets.csv', low_memory=False)
social_spambots_3_tweets = pd.read_csv('cresci-2017.csv/datasets_full.csv/social_spambots_3.csv/tweets.csv', low_memory=False)
social_spambots_2_tweets = pd.read_csv('cresci-2017.csv/datasets_full.csv/social_spambots_2.csv/tweets.csv', low_memory=False)
russian_troll_tweets = pd.read_csv('russian-troll-tweets/tweets.csv', low_memory=False)

In [25]:
a = genuine_accounts_tweets
b= social_spambots_2_tweets
c= social_spambots_3_tweets

In [26]:
a['label']=int(0)
b['label']=int(1)
c['label']=int(1)

import random
a1 = a.sample(n=1000, random_state=1)
a2 = a.drop(index=a1.index).sample(n=1000, random_state=0)
b1 = b.sample(n=1000, random_state=1)
c1 = c.sample(n=1000, random_state=1)
b2 = c.drop(index=c1.index).sample(n=100, random_state=0)

In [27]:
ab=pd.concat([b1, a1, b2], axis=0)
ab_text = ab[['text', 'label']]
ab_text = ab_text.fillna('None')

ac=pd.concat([c, a2], axis=0)
ac_text=ac[['text', 'label']]
ac_text = ac_text.fillna('None')

In [28]:
from sklearn.utils import shuffle
ab_text=shuffle(ab_text)
ac_text=shuffle(ac_text)

In [29]:
# clean content: 
import spacy
#spacy.load('en')
#from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS
import re
import os
def cleanContent(contents):
    contents_new=[]
    contents_list = contents.tolist()
    tweets_list_new=[]
    for tweet in contents_list:
        if 'http' in tweet:
            index = tweet.rfind('http')
            tweet = tweet.replace(tweet[index:], '')
        tweets_list_new.append(tweet)
    for content in tweets_list_new:
        # Convert to lowercase
        content_low=content.lower()
        # Remove punctuation and any other non-alphabet characters
        content_low_pnt = re.sub(r'[^\w\s]', '', content_low)
        content_low_pnt = content_low_pnt.replace(os.linesep, " ")
        content_low_pnt = content_low_pnt.replace("\t", " ")
        content_low_pnt_alpha = re.sub('[^A-Za-z\s]', '', content_low_pnt)
        # Remove stopwords
        words=content_low_pnt_alpha.split()
        no_stop_words=[word for word in words if word not in STOP_WORDS]
        content=' '.join(no_stop_words)
        contents_new.append(content)
    return contents_new

In [30]:
ab_text.text=cleanContent(ab_text.text)
ac_text.text=cleanContent(ac_text.text)

In [31]:
ab_text_list=[]
for text in ab_text.text:
    ab_text_list.append(text)

In [32]:
ac_text_list=[]
for text in ac_text.text:
    ac_text_list.append(text)

In [33]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# SVM

In [34]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
text_clf = Pipeline([
     ('vect', CountVectorizer()),
     ('tfidf', TfidfTransformer()),
    ('clf', SVC(gamma='auto', max_iter=100, random_state=0))])

text_clf.fit(ab_text_list, ab_text.label)  

/Users/xinchunli/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...f',
  max_iter=100, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False))])

In [35]:
import numpy as np
predicted = text_clf.predict(ac_text_list)
np.mean(predicted == ac_text.label)            

0.0805705679799968

In [36]:
from sklearn import metrics
print(metrics.classification_report(ac_text.label, predicted))

metrics.confusion_matrix(ac_text.label, predicted)

              precision    recall  f1-score   support

           0       0.07      0.97      0.12    100000
           1       0.90      0.02      0.03   1418557

   micro avg       0.08      0.08      0.08   1518557
   macro avg       0.48      0.50      0.08   1518557
weighted avg       0.85      0.08      0.04   1518557



array([[  97267,    2733],
       [1393473,   25084]])

# Random Forest

In [37]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
text_clf = Pipeline([
     ('vect', CountVectorizer()),
     ('tfidf', TfidfTransformer()),
    ('clf', RandomForestClassifier(n_estimators=100, max_depth=20, criterion='entropy',
                                   random_state=0))])

text_clf.fit(ab_text_list, ab_text.label)  

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mators=100, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False))])

In [38]:
#coef = text_clf.coef_.ravel()
#top_positive_coefficients = np.argsort(coef)[-top_features:]
#plt.figure(figsize=(15, 5))
#colors = ['grey' if c < 0 else 'grey' for c in coef[top_positive_coefficients]]
#plt.bar(np.arange(top_features), coef[top_positive_coefficients], color=colors)
#feature_names = np.array(feature_names)
#plt.xticks(np.arange(0, top_features), feature_names[top_positive_coefficients], rotation=60, ha='center')
#plt.show()

In [39]:
import numpy as np
predicted = text_clf.predict(ac_text_list)
np.mean(predicted == ac_text.label)            

0.9356250703793141

In [40]:
from sklearn import metrics
print(metrics.classification_report(ac_text.label, predicted))

metrics.confusion_matrix(ac_text.label, predicted)

              precision    recall  f1-score   support

           0       0.96      0.02      0.05    100000
           1       0.94      1.00      0.97   1418557

   micro avg       0.94      0.94      0.94   1518557
   macro avg       0.95      0.51      0.51   1518557
weighted avg       0.94      0.94      0.91   1518557



array([[   2331,   97669],
       [     88, 1418469]])

# Gradient Boosting

In [41]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier
text_clf = Pipeline([
     ('vect', CountVectorizer()),
     ('tfidf', TfidfTransformer()),
    ('clf', GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=5, random_state=0))])

text_clf.fit(ab_text_list, ab_text.label)  

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...    subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False))])

In [42]:
import numpy as np
predicted = text_clf.predict(ac_text_list)
np.mean(predicted == ac_text.label)            

0.8214759143054887

In [43]:
from sklearn import metrics
print(metrics.classification_report(ac_text.label, predicted))

metrics.confusion_matrix(ac_text.label, predicted)

              precision    recall  f1-score   support

           0       0.25      0.84      0.38    100000
           1       0.99      0.82      0.90   1418557

   micro avg       0.82      0.82      0.82   1518557
   macro avg       0.62      0.83      0.64   1518557
weighted avg       0.94      0.82      0.86   1518557



array([[  83709,   16291],
       [ 254808, 1163749]])

# Logistic Regression

In [44]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
text_clf = Pipeline([
     ('vect', CountVectorizer()),
     ('tfidf', TfidfTransformer()),
    ('clf', LogisticRegression(penalty="l1",random_state=0))])

text_clf.fit(ab_text_list, ab_text.label)  

/Users/xinchunli/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...e, penalty='l1', random_state=0, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))])

In [45]:
import numpy as np
predicted = text_clf.predict(ac_text_list)
np.mean(predicted == ac_text.label)            

0.9428253269386662

In [46]:
from sklearn import metrics
print(metrics.classification_report(ac_text.label, predicted))

metrics.confusion_matrix(ac_text.label, predicted)

              precision    recall  f1-score   support

           0       0.54      0.84      0.66    100000
           1       0.99      0.95      0.97   1418557

   micro avg       0.94      0.94      0.94   1518557
   macro avg       0.77      0.89      0.81   1518557
weighted avg       0.96      0.94      0.95   1518557



array([[  83787,   16213],
       [  70610, 1347947]])